## Creating a logistic regresssion to predict absenteeism

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
data_preprocessed = pd.read_csv('preprocessed-file.csv')

In [3]:
data_preprocessed.head()

,Reason_type_1,Reason_type_2,Reason_type_3,Reason_type_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [4]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

### Declare the inputs and targets

In [6]:
data_preprocessed['Excessive absenteeism'] = targets

In [7]:
data_preprocessed.head()

,Reason_type_1,Reason_type_2,Reason_type_3,Reason_type_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [8]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Daily Work Load Average','Distance to Work','Day of the week'], axis = 1)

In [9]:
data_with_targets.shape

(700, 12)

In [10]:
data_with_targets.iloc[:, :-1]

,Reason_type_1,Reason_type_2,Reason_type_3,Reason_type_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,289,33,30,0,2,1
1,0,0,0,0,7,118,50,31,0,1,0
2,0,0,0,1,7,179,38,31,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0
4,0,0,0,1,7,289,33,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,179,40,22,1,2,0
696,1,0,0,0,5,225,28,24,0,1,2
697,1,0,0,0,5,330,28,25,1,0,0
698,0,0,0,1,5,235,32,25,1,0,0


In [11]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

### Scale the data

In [12]:
from sklearn.preprocessing import StandardScaler
absenteeism_scaler = StandardScaler()

In [13]:
absenteeism_scaler.fit(unscaled_inputs)

StandardScaler()

In [14]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [15]:
scaled_inputs.shape

(700, 11)

## Train test split

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

In [18]:
print(x_train.shape, y_train.shape)

(560, 11) (560,)


In [19]:
print(x_test.shape, y_test.shape)

(140, 11) (140,)


# Create the regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [21]:
reg = LogisticRegression()

### Train the model

In [22]:
reg.fit(x_train,y_train)

LogisticRegression()

In [23]:
reg.score(x_train,y_train)

0.7839285714285714

#### The weight and bias

In [24]:
reg.intercept_

array([-0.22375632])

In [25]:
reg.coef_

array([[ 2.0695214 ,  0.33581057,  1.56203942,  1.31277993,  0.18469755,
         0.69126151, -0.19828303,  0.32435951, -0.12665599,  0.37017705,
        -0.32535864]])

In [26]:
feature_name = unscaled_inputs.columns.values

In [27]:
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature name,Coefficient
0,Reason_type_1,2.069521
1,Reason_type_2,0.335811
2,Reason_type_3,1.562039
3,Reason_type_4,1.312780
4,Month Value,0.184698
5,Transportation Expense,0.691262
6,Age,-0.198283
7,Body Mass Index,0.324360
8,Education,-0.126656
9,Children,0.370177


In [28]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-0.223756
1,Reason_type_1,2.069521
2,Reason_type_2,0.335811
3,Reason_type_3,1.562039
4,Reason_type_4,1.312780
5,Month Value,0.184698
6,Transportation Expense,0.691262
7,Age,-0.198283
8,Body Mass Index,0.324360
9,Education,-0.126656


In [29]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-0.223756,0.799510
1,Reason_type_1,2.069521,7.921031
2,Reason_type_2,0.335811,1.399074
3,Reason_type_3,1.562039,4.768536
4,Reason_type_4,1.312780,3.716491
5,Month Value,0.184698,1.202855
6,Transportation Expense,0.691262,1.996232
7,Age,-0.198283,0.820138
8,Body Mass Index,0.324360,1.383144
9,Education,-0.126656,0.881037


In [30]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature name,Coefficient,Odds_ratio
1,Reason_type_1,2.069521,7.921031
3,Reason_type_3,1.562039,4.768536
4,Reason_type_4,1.312780,3.716491
6,Transportation Expense,0.691262,1.996232
10,Children,0.370177,1.447991
2,Reason_type_2,0.335811,1.399074
8,Body Mass Index,0.324360,1.383144
5,Month Value,0.184698,1.202855
9,Education,-0.126656,0.881037
7,Age,-0.198283,0.820138


### Testing

In [31]:
reg.score(x_test,y_test)

0.7357142857142858

In [32]:
# predict
y_hat_test = reg.predict(x_test)
y_hat_test

array([0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0])

In [33]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.70804629, 0.29195371],
       [0.57178351, 0.42821649],
       [0.39895299, 0.60104701],
       [0.78714111, 0.21285889],
       [0.06691516, 0.93308484],
       [0.31109284, 0.68890716],
       [0.28582378, 0.71417622],
       [0.08093986, 0.91906014],
       [0.79926793, 0.20073207],
       [0.7496687 , 0.2503313 ],
       [0.46772305, 0.53227695],
       [0.18449682, 0.81550318],
       [0.04086632, 0.95913368],
       [0.75596678, 0.24403322],
       [0.23709315, 0.76290685],
       [0.54140798, 0.45859202],
       [0.53354387, 0.46645613],
       [0.52039975, 0.47960025],
       [0.40507469, 0.59492531],
       [0.02759587, 0.97240413],
       [0.70148836, 0.29851164],
       [0.78714111, 0.21285889],
       [0.40545618, 0.59454382],
       [0.40545618, 0.59454382],
       [0.17391667, 0.82608333],
       [0.75362875, 0.24637125],
       [0.48817233, 0.51182767],
       [0.88053255, 0.11946745],
       [0.13059235, 0.86940765],
       [0.78714111, 0.21285889],
       [0.

In [34]:
predicted_proba.shape

(140, 2)

In [35]:
predicted_proba[:,1]

array([0.29195371, 0.42821649, 0.60104701, 0.21285889, 0.93308484,
       0.68890716, 0.71417622, 0.91906014, 0.20073207, 0.2503313 ,
       0.53227695, 0.81550318, 0.95913368, 0.24403322, 0.76290685,
       0.45859202, 0.46645613, 0.47960025, 0.59492531, 0.97240413,
       0.29851164, 0.21285889, 0.59454382, 0.59454382, 0.82608333,
       0.24637125, 0.51182767, 0.11946745, 0.86940765, 0.21285889,
       0.37752575, 0.69562155, 0.71104836, 0.55842034, 0.21285889,
       0.56294675, 0.20932611, 0.81013019, 0.41289739, 0.63001412,
       0.20415726, 0.42151589, 0.22731649, 0.10486653, 0.85723233,
       0.65470153, 0.70629762, 0.29195371, 0.21207006, 0.19572289,
       0.56549874, 0.07774102, 0.68890716, 0.26970997, 0.85980383,
       0.45421224, 0.93050906, 0.21788125, 0.08531744, 0.08952364,
       0.70559435, 0.6774947 , 0.28721976, 0.86078009, 0.18989327,
       0.27063956, 0.01404469, 0.20932611, 0.81292865, 0.28500657,
       0.20932611, 0.06875653, 0.92859838, 0.47793046, 0.63865

### Model Evaluation 

In [36]:
confusion_matrix(y_test,y_hat_test)

array([[57, 17],
       [20, 46]], dtype=int64)

In [37]:
accuracy_score(y_test, y_hat_test)

0.7357142857142858

In [38]:
print(classification_report(y_test, y_hat_test))

              precision    recall  f1-score   support

           0       0.74      0.77      0.75        74
           1       0.73      0.70      0.71        66

    accuracy                           0.74       140
   macro avg       0.74      0.73      0.73       140
weighted avg       0.74      0.74      0.74       140



### Save the model

In [39]:
import pickle

In [40]:
with open('model', 'wb') as file:
    pickle.dump(reg,file)

In [41]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)